In [1]:
import pandas as pd
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt

import hillmaker as hm
#from hillmaker import bydatetime
#from hillmaker import hillpylib
from pandas import Timestamp

# Let's check what version of pandas, numpy and matplotlib we are using
print ("pandas version ", pd.__version__)
print ("numpy version ", np.version.version)
print ("matplotlib version ", mp.__version__)

pandas version  0.16.2
numpy version  1.9.2
matplotlib version  1.4.3


In [ ]:
hm

In [3]:
# Set up autoreloading of modules so that I can debug code in external files
%load_ext autoreload
%autoreload 2

In [ ]:
file_stopdata = 'data/ShortStay.csv'
df = pd.read_csv(file_stopdata, parse_dates=['InRoomTS','OutRoomTS'])

In [36]:
scenario_name = 'sstest_60'
in_fld_name = 'InRoomTS'
out_fld_name = 'OutRoomTS'
cat_fld_name = 'PatType'
start_analysis = '1/2/1996'
end_analysis = '3/31/1996 23:45'
bin_size_mins = 60

# This next field wasn't in original Hillmaker. Use it to specify the name to use for the overall totals.
tot_fld_name = 'Total'

## Convert string dates to actual datetimes
start_analysis_dt = pd.Timestamp(start_analysis)
end_analysis_dt = pd.Timestamp(end_analysis)




In [13]:
df.head()

,PatID,InRoomTS,OutRoomTS,PatType
0,1,1996-01-01 07:44:00,1996-01-01 08:50:00,IVT
1,2,1996-01-01 08:28:00,1996-01-01 09:20:00,IVT
2,3,1996-01-01 11:44:00,1996-01-01 13:30:00,MYE
3,4,1996-01-01 11:51:00,1996-01-01 12:55:00,CAT
4,5,1996-01-01 12:10:00,1996-01-01 13:00:00,IVT


In [37]:
bydt_df = bydatetime.make_bydatetime(df,
                                     in_fld_name,
                                     out_fld_name,
                                     cat_fld_name,
                                     start_analysis_dt,
                                     end_analysis_dt,
                                     tot_fld_name,
                                     bin_size_mins)

rng_bydt created: 0.0006284789997152984
found unique categories: 0.004088978999789106
Seeded bydatetime DataFrame created: 0.0356079929997577
dayofweek, bin_of_day, bin_of_week computed: 0.34738577700045425
Multi-index on bydatetime DataFrame created: 0.3504454660005649
Multi-index fully lexsorted: 0.3538809880001281
Num inner: 19793
Done processing 19793 stop recs: 14.286970832999941


In [17]:
bydt_df.head()

category            datetime  arrivals  \
category datetime                                                     
ART      1996-01-02 00:00:00      ART 1996-01-02 00:00:00         0   
         1996-01-02 01:00:00      ART 1996-01-02 01:00:00         0   
         1996-01-02 02:00:00      ART 1996-01-02 02:00:00         0   
         1996-01-02 03:00:00      ART 1996-01-02 03:00:00         0   
         1996-01-02 04:00:00      ART 1996-01-02 04:00:00         0   

                              departures  occupancy  day_of_week  bin_of_day  \
category datetime                                                              
ART      1996-01-02 00:00:00           0          0            1           0   
         1996-01-02 01:00:00           0          0            1           1   
         1996-01-02 02:00:00           0          0            1           2   
         1996-01-02 03:00:00           0          0            1           3   
         1996-01-02 04:00:00           0          0            1           4   

                              bin_of_week  
category datetime                          
ART      1996-01-02 00:00:00           24  
         1996-01-02 01:00:00           25  
         1996-01-02 02:00:00           26  
         1996-01-02 03:00:00           27  
         1996-01-02 04:00:00           28

In [38]:
file_bydt_csv = 'data/bydate_' + scenario_name + '.csv'
bydt_df.to_csv(file_bydt_csv)

With this data frame we can compute all kinds of interesting summary statistics by category, by day of week and time of day. To facilitate this type of "group by" analysis, **pandas** takes what is known as the Split-Apply-Combine approach. The [pandas documentation has a nice discussion](http://pandas.pydata.org/pandas-docs/dev/groupby.html) of this. To really understand split-apply-combine, [check out the article](http://www.jstatsoft.org/v40/i01) by [Hadley Wickham](http://had.co.nz/) who created the **plyr** package for [R](http://www.r-project.org/). I also created a tutorial on [Getting started with Python (with pandas and matplotlib) for group by analysis](http://hselab.org/machinery/content/getting-started-python-pandas-and-matplotlib-group-analysis) that covers some of the basics. A [companion tutorial shows how to do the same analysis using R](http://hselab.org/machinery/content/getting-started-r-plyr-and-ggplot2-group-analysis) instead of Python.

Pandas provides a `GroupBy` object to facilitate computing aggregate statistics by grouping fields. 

In [18]:
# Create a GroupBy object for the summary stats    
bydt_dfgrp1 = bydt_df.groupby(['category','bin_of_week'])

In [19]:
# Having a group by object makes it easy to compute statistics such as the mean of all of the fields other than the grouping fields.
# You'll see that the result is simply another DataFrame.
bydt_dfgrp1.mean()

arrivals  departures  occupancy  day_of_week  bin_of_day
category bin_of_week                                                          
ART      0            0.000000    0.000000   0.000000            0           0
         1            0.000000    0.000000   0.000000            0           1
         2            0.000000    0.000000   0.000000            0           2
         3            0.000000    0.000000   0.000000            0           3
         4            0.000000    0.000000   0.000000            0           4
         5            1.000000    0.000000   0.309722            0           5
         6            5.916667    0.000000   4.154167            0           6
         7            3.583333    4.916667   6.440278            0           7
         8            4.166667    3.833333   5.390278            0           8
         9            2.916667    4.083333   5.215278            0           9
         10           3.166667    3.000000   4.443056            0          10
         11           2.750000    3.333333   4.855556            0          11
         12           3.000000    3.500000   3.987500            0          12
         13           2.000000    2.666667   3.558333            0          13
         14           0.583333    2.583333   2.095833            0          14
         15           0.333333    1.000000   0.668056            0          15
         16           0.000000    0.333333   0.319444            0          16
         17           0.083333    0.250000   0.090278            0          17
         18           0.000000    0.000000   0.000000            0          18
         19           0.000000    0.000000   0.000000            0          19
         20           0.000000    0.000000   0.000000            0          20
         21           0.000000    0.000000   0.000000            0          21
         22           0.000000    0.000000   0.000000            0          22
         23           0.000000    0.000000   0.000000            0          23
         24           0.000000    0.000000   0.000000            1           0
         25           0.000000    0.000000   0.000000            1           1
         26           0.000000    0.000000   0.000000            1           2
         27           0.000000    0.000000   0.000000            1           3
         28           0.000000    0.000000   0.000000            1           4
         29           0.000000    0.000000   0.000000            1           5
...                        ...         ...        ...          ...         ...
OTH      138          0.000000    0.000000   0.000000            5          18
         139          0.000000    0.000000   0.000000            5          19
         140          0.000000    0.000000   0.000000            5          20
         141          0.000000    0.000000   0.000000            5          21
         142          0.000000    0.000000   0.000000            5          22
         143          0.000000    0.000000   0.000000            5          23
         144          0.000000    0.000000   0.000000            6           0
         145          0.000000    0.000000   0.000000            6           1
         146          0.000000    0.000000   0.000000            6           2
         147          0.000000    0.000000   0.000000            6           3
         148          0.000000    0.000000   0.000000            6           4
         149          0.000000    0.000000   0.000000            6           5
         150          0.000000    0.000000   0.000000            6           6
         151          0.000000    0.000000   0.000000            6           7
         152          0.000000    0.000000   0.000000            6           8
         153          0.000000    0.000000   0.000000            6           9
         154          0.000000    0.000000   0.000000            6          10
         155          0.000000    0.000000   0.000000            6      

In [20]:
# Let's explore some of the means.
bydt_dfgrp1.mean()[100:120]

arrivals  departures  occupancy  day_of_week  bin_of_day
category bin_of_week                                                          
ART      100          0.000000    0.000000   0.000000            4           4
         101          0.769231    0.000000   0.167949            4           5
         102          7.153846    0.000000   3.471795            4           6
         103          3.692308    5.538462   6.602564            4           7
         104          4.307692    4.153846   5.932051            4           8
         105          3.692308    4.461538   5.807692            4           9
         106          4.769231    3.923077   6.028205            4          10
         107          3.230769    3.846154   6.094872            4          11
         108          3.769231    4.461538   4.975641            4          12
         109          2.615385    3.461538   4.260256            4          13
         110          0.846154    3.000000   2.921795            4          14
         111          0.538462    2.000000   0.970513            4          15
         112          0.461538    0.307692   0.635897            4          16
         113          0.000000    0.692308   0.230769            4          17
         114          0.000000    0.000000   0.000000            4          18
         115          0.000000    0.000000   0.000000            4          19
         116          0.000000    0.000000   0.000000            4          20
         117          0.000000    0.000000   0.000000            4          21
         118          0.000000    0.000000   0.000000            4          22
         119          0.000000    0.000000   0.000000            4          23

Now that we've seen how the a `GroupBy` object works, let's see how we can compute a whole bunch of summary statistics at once. Specifically we want to compute the mean, standard deviation, min, max and several percentiles. First let's create a slightly different `GroupBy` object.

Now let's define a function that will return a bunch of statistics in a dictionary for a column of data.

In [23]:
def get_occstats(group, stub=''):
    return {stub+'count': group.count(), stub+'mean': group.mean(), 
            stub+'min': group.min(),
            stub+'max': group.max(), 'stdev': group.std(), 
            stub+'p50': group.quantile(0.5), stub+'p55': group.quantile(0.55),
            stub+'p60': group.quantile(0.6), stub+'p65': group.quantile(0.65),
            stub+'p70': group.quantile(0.7), stub+'p75': group.quantile(0.75),
            stub+'p80': group.quantile(0.8), stub+'p85': group.quantile(0.85),
            stub+'p90': group.quantile(0.9), stub+'p95': group.quantile(0.95),
            stub+'p975': group.quantile(0.975), 
            stub+'p99': group.quantile(0.99)}

In [39]:
bydt_dfgrp2 = bydt_df.groupby(['category','day_of_week','bin_of_day'])

Now we can use the apply function to apply the get_occstats() function to a data series. We'll create separate output data series for occupancy, arrivals and departures.

In [40]:
occ_stats = bydt_dfgrp2['occupancy'].apply(get_occstats)
arr_stats = bydt_dfgrp2['arrivals'].apply(get_occstats)
dep_stats = bydt_dfgrp2['departures'].apply(get_occstats)

So, what is `occ_stats`?

In [26]:
type(occ_stats)

pandas.core.series.Series

It's a pandas `Series` object. What does its index look like?

In [27]:
occ_stats.index

MultiIndex(levels=[['ART', 'CAT', 'IVT', 'MYE', 'OTH'], [0, 1, 2, 3, 4, 5, 6], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], ['count', 'max', 'mean', 'min', 'p50', 'p55', 'p60', 'p65', 'p70', 'p75', 'p80', 'p85', 'p90', 'p95', 'p975', 'p99', 'stdev']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,

Notice it's a `MultiIndex` with 4 levels: category, dayofweek, binofday, statistic. It would be nice to "un-pivot" the statistic from the index and have it correspond to a set of columns. That's what `unstack()` will do. It will leave us with a `DataFrame` with all of the statistics as columns and a 3 level multi-index of category, dayofweek and binofday. Perfect for plotting.

In [29]:
occ_stats.unstack()[100:125]

count       max      mean       min  \
category day_of_week bin_of_day                                        
ART      4           4              13  0.000000  0.000000  0.000000   
                     5              13  0.866667  0.167949  0.000000   
                     6              13  6.033333  3.471795  1.283333   
                     7              13  8.283333  6.602564  2.750000   
                     8              13  8.400000  5.932051  2.166667   
                     9              13  7.433333  5.807692  3.100000   
                     10             13  9.700000  6.028205  3.383333   
                     11             13  8.400000  6.094872  4.083333   
                     12             13  8.000000  4.975641  3.366667   
                     13             13  7.200000  4.260256  2.300000   
                     14             13  5.933333  2.921795  1.166667   
                     15             13  3.233333  0.970513  0.000000   
                     16             13  2.000000  0.635897  0.000000   
                     17             13  0.833333  0.230769  0.000000   
                     18             13  0.000000  0.000000  0.000000   
                     19             13  0.000000  0.000000  0.000000   
                     20             13  0.000000  0.000000  0.000000   
                     21             13  0.000000  0.000000  0.000000   
                     22             13  0.000000  0.000000  0.000000   
                     23             13  0.000000  0.000000  0.000000   
         5           0              13  0.000000  0.000000  0.000000   
                     1              13  0.000000  0.000000  0.000000   
                     2              13  0.000000  0.000000  0.000000   
                     3              13  0.000000  0.000000  0.000000   
                     4              13  0.000000  0.000000  0.000000   

                                      p50       p55       p60       p65  \
category day_of_week bin_of_day                                           
ART      4           4           0.000000  0.000000  0.000000  0.000000   
                     5           0.000000  0.030000  0.090000  0.210000   
                     6           3.333333  3.503333  3.693333  3.923333   
                     7           7.383333  7.513333  7.626667  7.706667   
                     8           6.383333  6.483333  6.580000  6.670000   
                     9           6.016667  6.206667  6.340000  6.360000   
                     10          5.933333  6.083333  6.203333  6.263333   
                     11          5.666667  6.176667  6.520000  6.530000   
                     12          4.800000  4.920000  5.070000  5.280000   
                     13          3.816667  4.356667  4.760000  4.890000   
                     14          2.683333  2.703333  2.800000  3.050000   
                     15          0.916667  0.966667  1.066667  1.266667   
                     16          0.516667  0.526667  0.540000  0.560000   
                     17          0.000000  0.050000  0.116667  0.216667   
                     18          0.000000  0.000000  0.000000  0.000000   
                     19          0.000000  0.000000  0.000000  0.000000   
                     20          0.000000  0.000000  0.000000  0.000000   
                     21          0.000000  0.000000  0.000000  0.000000   
                     22          0.000000  0.000000  0.000000  0.000000   
                     23          0.000000  0.000000  0.000000  0.000000   
         5           0           0.000000  0.000000  0.000000  0.000000   
                     1           0.000000  0.000000  0.000000  0.000000   
                     2           0.000000  0.000000  0.000000  0.000000   
                     3           0.000000  0.000000  0.000000  0.000000   
                     4           0.000000  0.000000  0.000000  0.000000   

                                      p70       p75       p80 

In [41]:
occ_stats_summary = occ_stats.unstack()
arr_stats_summary = arr_stats.unstack()
dep_stats_summary = dep_stats.unstack()

In [32]:
print (occ_stats_summary[200:220].values) # Let's peek into the middle of the table.

[[  1.30000000e+01   5.00000000e-01   8.33333333e-02   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   1.00000000e-01   2.16666667e-01
    3.66666667e-01   4.50000000e-01   4.75000000e-01   4.90000000e-01
    1.73472167e-01]
 [  1.30000000e+01   2.35000000e+00   5.34615385e-01   0.00000000e+00
    4.66666667e-01   4.76666667e-01   4.86666667e-01   4.96666667e-01
    5.60000000e-01   6.50000000e-01   6.90000000e-01   7.83333333e-01
    9.83333333e-01   1.57000000e+00   1.96000000e+00   2.19400000e+00
    6.33642525e-01]
 [  1.30000000e+01   6.01666667e+00   3.26410256e+00   1.25000000e+00
    3.10000000e+00   3.26000000e+00   3.41333333e+00   3.55333333e+00
    3.63333333e+00   3.68333333e+00   3.78333333e+00   4.11333333e+00
    4.90333333e+00   5.50666667e+00   5.76166667e+00   5.91466667e+00
    1.26978502e+00]
 [  1.30000000e+01   9.55000000e+00   5.77179487e+00   3.43333333e+00
    5.43333333e+00   5.4733333

Let's fire these guys out to csv files so we can check them out and maybe play with them in spreadsheet. 

In [33]:
file_occ_csv = 'data/occ_stats_' + scenario_name + '.csv'
file_arr_csv = 'data/arr_stats_' + scenario_name + '.csv'
file_dep_csv = 'data/dep_stats_' + scenario_name + '.csv'

occ_stats_summary.to_csv('data/occ_stats_summary.csv')
arr_stats_summary.to_csv('data/arr_stats_summary.csv')
dep_stats_summary.to_csv('data/dep_stats_summary.csv')

# Put it all together

Below I've strung together all the pieces to do an entire Hillmaker run. Change inputs as needed (e.g. scenario_name and associated parameter values) and run all the cells below. You can skip rereading the main input file if that isn't changing.

## Read main stop data file

In [42]:
file_stopdata = 'data/ShortStay.csv'
df = pd.read_csv(file_stopdata, parse_dates=['InRoomTS','OutRoomTS'])

## Set input parameters

In [43]:
scenario_name = 'sstest_60'
in_fld_name = 'InRoomTS'
out_fld_name = 'OutRoomTS'
cat_fld_name = 'PatType'
start_analysis = '1/2/1996'
end_analysis = '3/31/1996 23:45'
bin_size_mins = 60

# This next field wasn't in original Hillmaker. Use it to specify the name to use for the overall totals.
tot_fld_name = 'Total'

## Convert string dates to actual datetimes
start_analysis_dt = pd.Timestamp(start_analysis)
end_analysis_dt = pd.Timestamp(end_analysis)


## Create the by datetime table

In [44]:
bydt_df = bydatetime.make_bydatetime(df,
                                     in_fld_name,
                                     out_fld_name,
                                     cat_fld_name,
                                     start_analysis_dt,
                                     end_analysis_dt,
                                     tot_fld_name,
                                     bin_size_mins)

rng_bydt created: 0.0009996110002248315
found unique categories: 0.005060685000898957
Seeded bydatetime DataFrame created: 0.03853595900000073
dayofweek, bin_of_day, bin_of_week computed: 0.3139086690007389
Multi-index on bydatetime DataFrame created: 0.3179300960000546
Multi-index fully lexsorted: 0.32116212800065114
Num inner: 19793
Done processing 19793 stop recs: 15.905689233000885


## Compute summary stats

In [23]:
def get_occstats(group, stub=''):
    return {stub+'count': group.count(), stub+'mean': group.mean(), 
            stub+'min': group.min(),
            stub+'max': group.max(), 'stdev': group.std(), 
            stub+'p50': group.quantile(0.5), stub+'p55': group.quantile(0.55),
            stub+'p60': group.quantile(0.6), stub+'p65': group.quantile(0.65),
            stub+'p70': group.quantile(0.7), stub+'p75': group.quantile(0.75),
            stub+'p80': group.quantile(0.8), stub+'p85': group.quantile(0.85),
            stub+'p90': group.quantile(0.9), stub+'p95': group.quantile(0.95),
            stub+'p975': group.quantile(0.975), 
            stub+'p99': group.quantile(0.99)}

In [45]:
bydt_dfgrp2 = bydt_df.groupby(['category','day_of_week','bin_of_day'])

occ_stats = bydt_dfgrp2['occupancy'].apply(get_occstats)
arr_stats = bydt_dfgrp2['arrivals'].apply(get_occstats)
dep_stats = bydt_dfgrp2['departures'].apply(get_occstats)

occ_stats_summary = occ_stats.unstack()
arr_stats_summary = arr_stats.unstack()
dep_stats_summary = dep_stats.unstack()



## Write summaries and by datetime out to CSV

In [46]:
file_bydt_csv = 'data/bydate_' + scenario_name + '.csv'
bydt_df.to_csv(file_bydt_csv)

file_occ_csv = 'data/occ_stats_' + scenario_name + '.csv'
file_arr_csv = 'data/arr_stats_' + scenario_name + '.csv'
file_dep_csv = 'data/dep_stats_' + scenario_name + '.csv'

occ_stats_summary.to_csv('data/occ_stats_summary.csv')
arr_stats_summary.to_csv('data/arr_stats_summary.csv')
dep_stats_summary.to_csv('data/dep_stats_summary.csv')